## DiskCacheライブラリの使い方
+ インメモリではなく, SSDやHDDにキャッシュを作成する
+ key-valueデータ構造
+ Cacheオブジェクトはスレッドセーフ(プロセスセーフ)
+ Cache操作はすべてアトミック
+ プロセスフォークに対応(Python Pickle Serializerを利用)

In [60]:
from diskcache import Cache

cachedir = "F:/DiskCacheTest/Cache"
cache = Cache(cachedir) # ディレクトリ指定がない場合, 一時ディレクトリが自動的に作成される.
cache.close()


withステートメントに対応

In [61]:
with Cache(cache.directory) as reference:
    reference.set('key', 'value')

In [62]:
print("cache.get('key')", cache.get('key')) # Automatically opens, but slower.

cache.get('key') value


PythonのDictionary型と同じ操作ができる

In [63]:
cache['my-key'] = 'my-value'
print(cache['my-key'])
del cache['my-key']
print(cache['my-key'])


my-value


KeyError: 'my-key'

時間制限付きキャッシュ

In [ ]:
from io import BytesIO
cache.set('my-key', BytesIO(b'my-value'), expire=5, read=True, tag='data') # expire_time : 5sec

result = cache.get('my-key', read=True, expire_time=True, tag=True)
reader, timestamp, tag = result
print(reader.read().decode())
print(type(timestamp).__name__)
print(tag)

my-value
float
data


時間制限を超過した場合

In [ ]:
from io import BytesIO
cache.set('my-key', BytesIO(b'my-value'), expire=5, read=True, tag='data') # expire_time : 5sec

from time import sleep

sleep(6) # 6sec

result = cache.get('my-key', read=True, expire_time=True, tag=True) # read=Trueでファイルオブジェクトとして認識する
reader, timestamp, tag = result # タプル
print(reader.read().decode())
print(type(timestamp).__name__)
print(tag)

AttributeError: 'NoneType' object has no attribute 'read'

キャッシュオブジェクトの生存時間を更新するには`touch()`メソッドを使用する

In [ ]:
cache.touch('my-key', expire=None)

False

In [ ]:
cache.touch('dose-not-exist', expire=1) # 1sec

False

+ PythonのSet型と同じ操作ができる
+ valueの数値は, `SQLite interger column (64bit-singed integers)`

In [ ]:
cache.add(b'test', 123)

True

In [ ]:
cache[b'test']

123

In [ ]:
cache.add(b'test', 456)

False

In [ ]:
cache[b'test'] # 123 not 456

123

値の加算increment`incr()`メソッドと減算decriment`decr()`メソッド 

In [ ]:
cache.incr(b'test') # 123 -> 124

124

In [ ]:
cache.decr(b'test', 24) # 124 -> 100

100

`default=None`を指定することで, 存在しないkeyに対して`incr()`と`decr()`でKeyErrorを投げることができる

In [ ]:
cache.incr('alice')

1

In [ ]:
cache.decr('bob', default=-9) # 下駄=-9

-10

In [ ]:
cache.incr('carol', default=None)

KeyError: 'carol'

`delete()`, `get()`, `pop()`操作

In [ ]:
cache.pop('alice')

1

In [ ]:
cache.pop('dave', default='does not exist')

'does not exist'

In [ ]:
cache.set('dave', 0, expire=None, tag='admin')

True

In [ ]:
cache.get('dave')

0

In [64]:
cache.set('dave', 0, expire=None, tag='admin')

True

In [65]:
result = cache.pop('dave', expire_time=True, tag=True) # key`dave`のvalueを取得&削除
value, timestampe, tag = result
print(value)
print(timestamp)
print(tag)

0
None
admin


In [ ]:
cache.get('dave') # valueがない

削除系 `clear()`, `reset()`, `expire()`, `evict()`

In [66]:
cache.clear()

2

In [67]:
cache.reset('cull_limit', 0) # Disable automatic evictions(立ち退き)

0

In [68]:
cache.get('cull_limit') # まだkey-valueは存在している

In [69]:
for num in range(10):
    _ = cache.set(num, num, expire=1e-9) # Expire immediately
len(cache)

10

In [70]:
list(cache)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

`expire()`関数でexpiredされたkeyをすべて削除する

In [71]:
import time
time.sleep(1) # 1sec
cache.expire()

10

+ `evict()`関数は, タグにマッチしたkey-valueをすべて削除する. defaultタグは`None`
+ タグは, interger, float, string, bytes, Noneがあり得る
+ envict()関数を高速化するために, `tag_index=True`でインデックスを作成できる.

In [84]:
from time import perf_counter_ns

cache.clear()

tp_start = perf_counter_ns()

for num in range(100):
    _ = cache.set(num, num, tag='odd' if num % 2 else 'even')
cache.evict('even')

tp_end = perf_counter_ns()

duration = (tp_end - tp_start) / 1000.0
print("duration[ms]", duration)

duration[ms] 7841.6


In [86]:
cachedir2 = "F:/DiskCacheTest/TagIndexCache"
cache_with_tag_index = Cache(cachedir2, tag_index=True)

cache_with_tag_index.clear()

tp_start = perf_counter_ns()

for num in range(100):
    _ = cache_with_tag_index.set(num, num, tag='odd' if num % 2 else 'even')
cache.evict('even') # 処理速度が上昇する?

tp_end = perf_counter_ns()

duration = (tp_end - tp_start) / 1000.0
print("duration[ms]", duration)

duration[ms] 9118.6


`tag_index`の作成と削除

In [74]:
cache.drop_tag_index()

In [75]:
cache.create_tag_index()